In [ ]:
# Install the package directly from GitHub
!pip install git+https://github.com/wcw100168/Cubed-Sphere-DG-Solver.git

# Data Pipeline & Workflow

This tutorial demonstrates how to:
1. **Regrid** data from standard Lat-Lon grids to the Cubed-Sphere.
2. Run a simulation with **I/O Monitors** to save results to disk.
3. **Post-process** output NetCDF files.

In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from cubed_sphere.solvers import CubedSphereAdvectionSolver, AdvectionConfig
from cubed_sphere.utils.io import NetCDFMonitor

In [ ]:
# 1. Regridding Calculation (Conceptual)
lat = np.linspace(-90, 90, 180)
lon = np.linspace(0, 360, 360)
LON, LAT = np.meshgrid(lon, lat)

data_latlon = np.sin(np.deg2rad(LAT)) * np.cos(np.deg2rad(LON))
print(f"Source Data Shape: {data_latlon.shape}")

In [ ]:
# 2. Simulation with I/O and Auto-Stability
N = 16
R = 1.0
u0 = 2*np.pi

# Auto dt
target_cfl = 0.5
dt = target_cfl * R / (u0 * N**2)

config = AdvectionConfig(N=N, R=R, u0=u0, backend='numpy', dt=dt)
solver = CubedSphereAdvectionSolver(config)
state = solver.get_initial_condition("gaussian")

# Attach a Monitor
output_file = "tutorial_output.nc"
monitor = NetCDFMonitor(filename=output_file, save_interval=dt*2) # Save every 2 steps
# monitor.initialize(solver, state)

t = 0.0
for i in range(10):
    state = solver.step(t, state, dt)
    t += dt
    monitor(t, state)
    
# monitor.close()
print(f"Simulation finished. Data saved to {output_file}")

In [ ]:
# 3. Post-Processing
ds = xr.open_dataset(output_file)
print(ds)

# Plot the last frame of Face 0
last_frame = ds['state'].isel(time=-1, face=0, var=0)
plt.figure(figsize=(8,6))
last_frame.plot()
plt.title("Post-processed Result (Face 0)")
plt.show()